In [29]:
from faker import Faker
import random
from datetime import datetime, timedelta

fake = Faker()

# Fonction pour générer une date aléatoire entre deux dates
def random_date(start_date, end_date):
    return start_date + timedelta(seconds=random.randint(0, int((end_date - start_date).total_seconds())))

# Fonction pour générer une requête INSERT pour un utilisateur
def generate_user_insert():
    username = fake.unique.user_name()
    while len(username) < 9:
        username = fake.unique.user_name()
    
    password = fake.user_name()
    while len(password) < 9:
        password = fake.password()
    return f"INSERT INTO user_table (firstname, lastname, email, username, password, created_at) VALUES ('{fake.first_name()}', '{fake.last_name()}', '{fake.unique.email()}', '{username}', '{password}', '{random_date(datetime(2023, 1, 1), datetime(2023, 6, 30))}');"

# Fonction pour générer une requête INSERT pour une vérification d'email
def generate_verification_insert(user_id, created_at):
    return f"INSERT INTO user_email_verification_table (user_id, verified_at) VALUES ({user_id}, '{random_date(created_at, datetime(2023, 6, 30))}');"

# Fonction pour générer une requête INSERT pour une session
def generate_session_insert(user_id, created_at, verification_date):
    return f"INSERT INTO session_table (user_id, connected_at) VALUES ({user_id}, '{random_date(max(created_at, verification_date), datetime(2023, 6, 30))}');"

# Écriture des requêtes dans le fichier SQL
with open('../part_1_2.sql', 'w') as file:
    # Insérer 100 utilisateurs
    nbVerifs = 0
    user_ids = list(range(1, 101))
    for user_id in user_ids:
        user_insert = generate_user_insert()
        file.write(user_insert + '\n')

        # Insérer au maximum 85 vérifications d'e-mails
        if (nbVerifs<85):
            verification_insert = generate_verification_insert(user_id, datetime(2023, 1, 1))
            file.write(verification_insert + '\n')
            nbVerifs+=1

            # Insérer 10 sessions pour chaque utilisateur
        for _ in range(10):
            session_insert = generate_session_insert(user_id, datetime(2023, 1, 1), datetime(2023, 1, 1))
            file.write(session_insert + '\n')
